In [ ]:
%config Completer.use_jedi = False

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.impute import KNNImputer
from eli5.sklearn import PermutationImportance
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix ,ConfusionMatrixDisplay
import pickle

In [ ]:
data = pd.read_csv('/kaggle/input/heart-disease-prediction-using-logistic-regression/framingham.csv')
data.head()

In [ ]:
print(data.head())
print("========================================================")
print(data.columns)
print("========================================================")
print(data.describe())
print("========================================================")
print(data.isnull().sum())
print("========================================================")
print(data.shape)
print("========================================================")
print(data.dtypes)

mencari data yg hilang

In [ ]:
data.isnull().sum()

mengecek apakah ada data yg tidak seimbang

In [ ]:
sns.displot(data.TenYearCHD)

Handling missing values & Feature Engineering.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(data.isnull())

Imputing the "education" feature.



In [ ]:
data["education"].describe()


In [ ]:
plt.figure()
sns.displot(data.education, kde=True)

In [ ]:
data["education_nan"] = np.where(data.education.isnull(),1,0)
data["education"].fillna(5, inplace=True)

Imputing the "cigsPerDay" feature.

In [ ]:
data.cigsPerDay.describe()

In [ ]:
plt.figure()
sns.displot(data.cigsPerDay, kde=True)

In [ ]:
data["cigsPerDay"].fillna(9, inplace=True)

Imputing the "BPMeds" feature.

In [ ]:
data.BPMeds.describe()

In [ ]:
plt.figure()
sns.displot(data.BPMeds, kde=True)

In [ ]:
data["BPMeds_nan"] = np.where(data.BPMeds.isnull(),1,0)
data["BPMeds"].fillna(0, inplace=True)

Imputing the "totChol" features

In [ ]:
data.totChol.describe()

In [ ]:
plt.figure()
sns.displot(data.totChol, kde=True)

In [ ]:
data["totChol_nan"] = np.where(data.totChol.isnull(),1,0)
data["totChol"].fillna(data.totChol.median(), inplace=True)

Imputing the "BMI" features

In [ ]:
data.BMI.describe()

In [ ]:
plt.figure()
sns.displot(data.BMI, kde=True)

In [ ]:
data["BMI"].fillna(data.BMI.mean(), inplace=True)

Imputing the "glucose" feature

In [ ]:
data.glucose.describe()

In [ ]:
plt.figure()
sns.displot(data.glucose, kde=True)

In [ ]:
data["glucose_nan"] = np.where(data.glucose.isnull(),1,0)
data["glucose"].fillna(data.glucose.median(), inplace=True)

imputing the "heartRate" feature

In [ ]:
data.heartRate.describe()

In [ ]:
plt.figure()
sns.displot(data.heartRate, kde=True)

In [ ]:
data["heartRate"].fillna(data.heartRate.median(), inplace=True)

pemodelan

In [ ]:
# membagi data ke fitur dan variabel yg di targetkan.
x = data.drop(columns="TenYearCHD")
y = data.TenYearCHD

# memperbaiki data yang tidak seimbang dengan data random oversampling .
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(x, y)

# Splitting the data into features and target variable.
train_x, test_x, train_y, test_y = train_test_split(X_resampled, y_resampled, test_size=0.25)

In [ ]:
# menyeimbangkan dataset
sns.displot(y_resampled)

In [ ]:
# membuat pipelies.

pipe1 = Pipeline([("robust_scalar", RobustScaler()),("std_scalar", StandardScaler()), ("logistic_regression", LogisticRegression())])

pipe2 = Pipeline([("robust_scalar", RobustScaler()),("std_scalar", StandardScaler()), ("KNN", KNeighborsClassifier())])

pipe3 = Pipeline([("robust_scalar", RobustScaler()),("std_scalar", StandardScaler()), ("svm", SVC())])

pipe4 = Pipeline([("robust_scalar", RobustScaler()),("std_scalar", StandardScaler()), ("XGboost", XGBClassifier())])

pipe5 = Pipeline([("robust_scalar", RobustScaler()),("std_scalar", StandardScaler()), ("decision_tree", DecisionTreeClassifier())])

pipe6 = Pipeline([("robust_scalar", RobustScaler()),("std_scalar", StandardScaler()), ("random_forest", RandomForestClassifier())])

In [ ]:
# menepatkan pipelines

pipelines = [pipe1, pipe2, pipe3, pipe4, pipe5, pipe6]

for pipe in pipelines:
    pipe.fit(train_x, train_y)

In [ ]:
# memprediksi

pred1 = pipe1.predict(test_x)
pred2 = pipe2.predict(test_x)
pred3 = pipe3.predict(test_x)
pred4 = pipe4.predict(test_x)
pred5 = pipe5.predict(test_x)
pred6 = pipe6.predict(test_x)

In [ ]:
# perbandingan nilai tiap pipeline dan memilih pipeline yang terbaik. More emphasis is given to recall as we really 
# want the false negative to be as low as possible.

print("Accuracy of Logistic_Regression", round(accuracy_score(test_y, pred1)*100, 2), "%")
print("Recall of Logistic_Regression", round(recall_score(test_y, pred1),2))
print("===================================================================")
print("Accuracy of KNN", round(accuracy_score(test_y, pred2)*100, 2), "%")
print("Recall of KNN", round(recall_score(test_y, pred2),2))
print("===================================================================")
print("Accuracy of SVC", round(accuracy_score(test_y, pred3)*100,2), "%")
print("Recall of SVC", round(recall_score(test_y, pred3),2))
print("===================================================================")
print("Accuracy of xgboost", round(accuracy_score(test_y, pred4)*100,2), "%")
print("Recall of xgboost", round(recall_score(test_y, pred4),2))
print("===================================================================")
print("Accuracy of decision_tree", round(accuracy_score(test_y, pred5)*100,2), "%")
print("Recall of decision_tree", round(recall_score(test_y, pred5),2))
print("===================================================================")
print("Accuracy of Random_forest", round(accuracy_score(test_y, pred6)*100,2), "%")
print("Recall of Random_forest", round(recall_score(test_y, pred6),2))

In [ ]:
# visualisasi confusion matrix. (Our aim is to reduce false negative)

cm = confusion_matrix(test_y, pred5)
display = ConfusionMatrixDisplay(confusion_matrix=cm)
display.plot()

In [ ]:
# Trying another method to reduce the unnecessary features

obj = SelectKBest(f_regression, k=4)
new_data = obj.fit_transform(x,y)

filter = obj.get_support()
feature = x.columns
final_f = feature[filter]
print(feature[filter])

In [ ]:
# Looking at the estimator's parameters

pipe5.get_params().keys()

In [ ]:
# Using randomized search cv to get the best parameter values

params = {
    'decision_tree__max_leaf_nodes'       : [1,2,4,6,8,12,15],
    #'decision_tree__max_features'         : ['auto', 'sqrt', 'log2'],
    'decision_tree__random_state'         : [42],
    'decision_tree__max_depth'            : [2, 3, 5, 10, 20],
    'decision_tree__min_samples_leaf'     : [5, 10, 20, 50, 100],
    'decision_tree__criterion'            : ["gini", "entropy"]
    }

from sklearn.model_selection import RandomizedSearchCV

final_model = RandomizedSearchCV(pipe5, param_distributions=params, n_iter=3, cv=3)

final_model.fit(train_x[final_f], train_y)

In [ ]:
# All the parameter values selected in the above process
final_model.best_params_

In [ ]:
# prediction with test data
prediction = final_model.predict(test_x[final_f])

In [ ]:
# visualizing the predicted values

plt.figure(figsize=(18,10))
sns.displot(prediction)

In [ ]:
# Looking at the metrics

print("Accuracy is", accuracy_score(test_y, prediction))
print("Recall is", recall_score(test_y, prediction))
print("Precision is", precision_score(test_y, prediction))

In [ ]:
# Visualizing confusion matrix

cm = confusion_matrix(test_y, prediction)
display = ConfusionMatrixDisplay(confusion_matrix=cm)
display.plot()

In [ ]:
# Manipulating our predicted values based on probabilities. This is done to reduce False Negative.
probalities = final_model.predict_proba(test_x[final_f])
prediction_prob = np.where(probalities>=0.30, 1, 0)[:,1]

# Visualizing Confusion matrix
cm = confusion_matrix(test_y, prediction_prob)
display_cm = ConfusionMatrixDisplay(confusion_matrix = cm)
display_cm.plot()